Tractor & Bin Size:
-----------------------------------------------------
Tractor Size in Crate: 6' x 11' x 4.5' : 66sqft
Bin Size: 40' x 60' x 18' : 2400sqft
Stack Size: 4 : Never stacked tractors higher than 4


t0 = Day 0
t_end = Day 7 to emulate a week



Reference for Reasonable Numbers:
------------------------------------------------------
units_in_container = 12
containers = 8

receiving = units_in_container*containers #96


Flow:

1. Give the system the bin size and unit size in square feet



Flow Truck Receiving Time:
1. Truck arrives and calls in arrival
2. Truck waits for assigned bay
3. Truck arrives at bay and hands over papers
4. Container and papers are checked for accuracy
5. Container is unloaded and received
6. Truck is given final documents
7. Truck Leaves

Flow Truck Shipping Time:
1. Truck arrives and calls in arrival
2. Truck waits for assigned bay
3. Truck arrives at bay and hands over papers
4. Papers are check for accuracy
5. Truck begins loading
6. Truck receives final go ahead
7. Truck parks and straps equipment on
8. Truck departs




In [13]:
import pint

import numpy
    
import scipy

import random

from modsim import *

In [17]:
foot = UNITS.foot
CK3510 = 6*foot*11*foot*4.5*foot #Tractor size
wh_Bin = 40*foot*60*foot*18*foot #Bin size
p = 3/7 #probability of receiving containers that day

units_in_bin = round(wh_Bin / CK3510)


### Create a timeframe of values to simulate realistic values of shipping/receiving to pump through the simulation
frame = TimeFrame(columns = ['Ship','Receive'])

for i in range(7):
    ship_value = random.randint(1, 25)  # Random value between 1 and 50 (inclusive)
    
    if flip(p):
        receive_value = random.randint(8,90)
    else:
        receive_value=0
    
    frame.row[i] = [ship_value,receive_value]

frame

,Ship,Receive
0,20,0
1,5,0
2,14,0
3,15,37
4,6,46
5,7,55
6,20,0


In [ ]:


def make_system(bin_size,unit_size,frame):
    """Make a system object for the WH Model
    
    bin_size: size of bin in feet^3
    unit_size: size of unit in feet^3
    
    returns: System object
    """
    init = State(C=50, ship=5, receive=5)

    t0 = 0
    t_end = 7
    
    full_bin = round(bin_size/unit_size)

    return System(init=init, t0=t0, t_end=t_end,
                  full_bin=full_bin)


def update_func(state, t, system):
    """Update the WH Model
    
    state: State object with C (Count of units in bin)
    t: time step
    system: system object with init, t0,t_end, full_bin
    
    returns: State object
    """
    count = state

    shipped = system.beta * i * s    
    received = system.gamma * i
    
    s -= infected
    i += infected - recovered
    r += recovered
    
    return State(S=s, I=i, R=r)


def run_simulation(system, update_func):
    """Runs a simulation of the system.
        
    system: System object
    update_func: function that updates state
    
    returns: TimeFrame
    """
    frame = TimeFrame(columns=system.init.index)
    frame.row[system.t0] = system.init
    
    for t in linrange(system.t0, system.t_end):
        frame.row[t+1] = update_func(frame.row[t], t, system)
    
    return frame

In [2]:
#VANILLA

def make_system(beta, gamma):
    """Make a system object for the SIR model.
    
    beta: contact rate in days
    gamma: recovery rate in days
    
    returns: System object
    """
    init = State(S=89, I=1, R=0)
    init /= sum(init)

    t0 = 0
    t_end = 7 * 14

    return System(init=init, t0=t0, t_end=t_end,
                  beta=beta, gamma=gamma)

def update_func(state, t, system):
    """Update the SIR model.
    
    state: State with variables S, I, R
    t: time step
    system: System with beta and gamma
    
    returns: State object
    """
    s, i, r = state

    infected = system.beta * i * s    
    recovered = system.gamma * i
    
    s -= infected
    i += infected - recovered
    r += recovered
    
    return State(S=s, I=i, R=r)


def run_simulation(system, update_func):
    """Runs a simulation of the system.
        
    system: System object
    update_func: function that updates state
    
    returns: TimeFrame
    """
    frame = TimeFrame(columns=system.init.index)
    frame.row[system.t0] = system.init
    
    for t in linrange(system.t0, system.t_end):
        frame.row[t+1] = update_func(frame.row[t], t, system)
    
    return frame


beta = 0.333
gamma = 0.25
system = make_system(beta, gamma)

results = run_simulation(system, update_func)
print(beta, gamma, calc_total_infected(results))



In [ ]:

state = State(avail_bin_space=10, =2, 
                  olin_empty=0, wellesley_empty=0)

def update(state, p1, p2):
    """Simulate one day at a time
    
    state: bin State object
    rec: arrival of units for receiving
    ship: departure of units for shipping
    """
    receive_unit(state)
    
    ship_unit(state)
        
def receive_unit(state):
    """Receive a CK3510
    
    state: bin State object
    """
    if state. 0:
        state.olin_empty += 1
        return
    state.olin -= 1
    state.wellesley += 1
    
def bike_to_olin(state):
    """Move one bike from Wellesley to Olin.
    
    state: bikeshare State object
    """
    if state.wellesley == 0:
        state.wellesley_empty += 1
        return
    state.wellesley -= 1
    state.olin += 1
    
def decorate_bikeshare():
    """Add a title and label the axes."""
    decorate(title='Olin-Wellesley Bikeshare',
             xlabel='Time step (min)', 
             ylabel='Number of bikes')
    
    
    
    
def run_simulation(p1, p2, num_steps):
    """Simulate the given number of time steps.
    
    p1: probability of an Olin->Wellesley customer arrival
    p2: probability of a Wellesley->Olin customer arrival
    num_steps: number of time steps
    """
    state = State(olin=10, wellesley=2, 
                  olin_empty=0, wellesley_empty=0)
                    
    for i in range(num_steps):
        step(state, p1, p2)
        
    return state